<a href="https://colab.research.google.com/github/rufous86/studies/blob/main/spark_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Скачаем необходимые наборы данных с kaggle  
инструкция по скачиванию данных в colab напрямую с kaggle ниже:  
https://www.kaggle.com/general/74235

In [1]:
! pip install -q kaggle
from google.colab import drive
drive.mount('/content/drive')
! mkdir ~/.kaggle
! cp '/content/drive/MyDrive/Colab Notebooks/kaggle.json' ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c 'riiid-test-answer-prediction'
! mkdir data
! unzip riiid-test-answer-prediction.zip -d data

Mounted at /content/drive
 99% 1.28G/1.29G [00:08<00:00, 165MB/s]
100% 1.29G/1.29G [00:09<00:00, 152MB/s]
Archive:  riiid-test-answer-prediction.zip
  inflating: data/example_sample_submission.csv  
  inflating: data/example_test.csv   
  inflating: data/lectures.csv       
  inflating: data/questions.csv      
  inflating: data/riiideducation/__init__.py  
  inflating: data/riiideducation/competition.cpython-37m-x86_64-linux-gnu.so  
  inflating: data/train.csv          


In [2]:
! pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 45 kB/s 
     |████████████████████████████████| 199 kB 56.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=4d1087bd9f28940d9c94e74bc82bf705d97ee0beffe78d6b0dcc39a51ec87440
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
spark

In [9]:
df = spark.read.csv('data/train.csv', header=True, inferSchema=True)

In [10]:
df.printSchema()

root
 |-- row_id: integer (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- content_id: integer (nullable = true)
 |-- content_type_id: integer (nullable = true)
 |-- task_container_id: integer (nullable = true)
 |-- user_answer: integer (nullable = true)
 |-- answered_correctly: integer (nullable = true)
 |-- prior_question_elapsed_time: double (nullable = true)
 |-- prior_question_had_explanation: boolean (nullable = true)



In [11]:
df.show()

+------+---------+-------+----------+---------------+-----------------+-----------+------------------+---------------------------+------------------------------+
|row_id|timestamp|user_id|content_id|content_type_id|task_container_id|user_answer|answered_correctly|prior_question_elapsed_time|prior_question_had_explanation|
+------+---------+-------+----------+---------------+-----------------+-----------+------------------+---------------------------+------------------------------+
|     0|        0|    115|      5692|              0|                1|          3|                 1|                       null|                          null|
|     1|    56943|    115|      5716|              0|                2|          2|                 1|                    37000.0|                         false|
|     2|   118363|    115|       128|              0|                0|          0|                 1|                    55000.0|                         false|
|     3|   131167|    115|  

In [53]:
df.to_pandas_on_spark().isna().sum()

row_id                                  0
timestamp                               0
user_id                                 0
content_id                              0
content_type_id                         0
task_container_id                       0
user_answer                             0
answered_correctly                      0
prior_question_elapsed_time       2351538
prior_question_had_explanation     392506
dtype: int64

Посмотрим, сколько в нашей таблице пустых значений

In [63]:
df = df.dropna()
df.pandas_api().isna().sum()

row_id                            0
timestamp                         0
user_id                           0
content_id                        0
content_type_id                   0
task_container_id                 0
user_answer                       0
answered_correctly                0
prior_question_elapsed_time       0
prior_question_had_explanation    0
dtype: int64

В таблице преобладают данные численного типа. Исключение составляет столбец prior_question_had_explanation.  
Посмотрим, какие уникальные значения составляют этот столбец. Также глянем на баланс этих значений

In [ ]:
df_train['prior_question_had_explanation'].value_counts(True)

True    0.885955
0       0.114045
Name: prior_question_had_explanation, dtype: float64

Видим, что в столбце присутствуют два значения True и 0 (часть из них до применения функции fillna была False, часть была NaN). Приведём столбец prior_question_had_explanation к целочисленному типу

In [ ]:
df_train['prior_question_had_explanation'] = df_train['prior_question_had_explanation'].astype('int8')

In [ ]:
df_train.isna().sum()

row_id                            0
timestamp                         0
user_id                           0
content_id                        0
content_type_id                   0
task_container_id                 0
user_answer                       0
answered_correctly                0
prior_question_elapsed_time       0
prior_question_had_explanation    0
dtype: int64

Проанализируем характеристики, влияющие на успеваемость студентов. Так как фактически данные об успеваемости у нас отсутствуют, условно за успеваемость будут выступать правильно данные ответы.      
Сначала сохраним колонку answered_correctly в переменную target. Это будет наша целевая переменная  
Затем рассчитаем коэффициент корреляции целевой переменной с каждой из остальных характеристик

In [ ]:
target = df_train.pop('answered_correctly')
df_train.corrwith(target)

row_id                           -0.000530
timestamp                         0.023392
user_id                          -0.000533
content_id                       -0.197708
content_type_id                  -0.437686
task_container_id                 0.060067
user_answer                       0.126696
prior_question_elapsed_time       0.068712
prior_question_had_explanation    0.257143
dtype: float64